In [ ]:
import pandas as pd
import altair as alt

In [ ]:
profile= pd.read_csv(
    #'Results/Batch4/CaseB/dispatch_10_1_2_price_improved.csv',
    #'Results/Batch4/CaseB/dispatch_10_1_2_price_limit.csv',
    'Results/Batch5/CaseB/co2_3_1_2.csv',
    #'Results/Batch5/CaseB/price_3_1_2.csv',

    sep= ',',
    index_col=False,
)
#profile['SoC']= ((profile['SoC']/20000)*1000).astype('float')
profile['SoC']= ((profile['SoC']/6000)*100).astype('float')

# p_solar=    10
# p_bess=     1
# t_bess=     2
# e_bess=     p_bess * t_bess
p_solar=    9
p_bess=     3
t_bess=     2
e_bess=     p_bess * t_bess

efficiency_charge=      0.98
efficiency_discharge=   0.96
efficiency_inverter=    0.97

week= 24

#profile.info()
#profile[47:96]

In [ ]:
profile['P_PV_to_Grid']=    profile['P_PV_to_Grid']     * efficiency_inverter
profile['P_PV_to_Load']=    profile['P_PV_to_Load']     * efficiency_inverter
profile['P_PV_to_BESS']=    profile['P_PV_to_BESS']     * efficiency_charge     * efficiency_inverter
profile['P_BESS_to_Grid']=  profile['P_BESS_to_Grid']   * efficiency_discharge  * efficiency_inverter
profile['P_BESS_to_Load']=  profile['P_BESS_to_Load']   * efficiency_discharge  * efficiency_inverter
profile['P_Grid_to_BESS']=  profile['P_Grid_to_BESS']   * efficiency_charge     * efficiency_inverter

profile= profile[:8760]
#profile

In [ ]:
price = pd.read_csv('Data/PriceCurve_SE3_2021.csv', sep = ';')
co2_pro = pd.read_csv('Data/production_emissions.csv')
co2_con = pd.read_csv('Data/consumption_emissions.csv')
pv = pd.read_csv('Data/pv_sam.csv')
load = pd.read_csv('Data/LoadCurve.csv', sep = ';')

data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

#Converting from MWh to kWh
data['Load']= (data['Load'] * 1000)
#solar data is already in kW, but needs to be multiplied by the scaling factor
data['solar_PV']= (data['solar_PV']) * p_solar
#data['Price']= (data['Price'] / 1000)
# CO_2_eq is also already in kWh
#data['CO_2_eq']= (data['CO_2_eq'] / 1000)

data['Hour']= (data['Hour']).astype('int')

#data.head(48)
data= data.drop(data.index[0])
data= ((data.reset_index()).drop(columns= {'Hour'})).rename(columns= {'index': 'Hour'})
data.head(10)

In [ ]:
# Select only the columns of interest
columns_of_interest = ['Hour','P_PV_to_Load','P_PV_to_Grid','P_PV_to_BESS','P_PV_curtailment','P_BESS_to_Load','P_Grid_to_Load']
subset_data_2= profile[columns_of_interest]
subset_data_2= subset_data_2.fillna(0)  

# Melt the DataFrame to long format
melted_data = subset_data_2.melt(id_vars=['Hour'], var_name='source', value_name='value')

alt.Chart(melted_data[4030:4201]).mark_area().encode(
    x= alt.X('Hour'),
    y= alt.Y(
        'value:Q', stack= 'zero'
    ),
    color= alt.Color(
        'source:N'
    )
)

In [ ]:
subset_data_2

In [ ]:
subset_data_2.info()

In [ ]:
melted_data[4030:4201]['source'].unique()

In [ ]:
data[:8760].describe()

In [ ]:
print(f'Price Variance: {data[:8760].Price.var()}')
print(f'Price Mean Value: {data[:8760].Price.mean()}')
print(f'CO2eq Variance: {data[:8760].CO_2_eq.var()}')
print(f'CO2eq Mean Value: {data[:8760].CO_2_eq.mean()}')
print('-'*50)
print(f'Price CoV: {data[:8760].Price.var()/data[:8760].Price.mean()}')
print(f'CO2eq CoV: {data[:8760].CO_2_eq.var()/data[:8760].CO_2_eq.mean()}')
print('-'*50)
print(f'Price Standard Deviation: {data[:8760].Price.std()}')
print(f'CO2eq Standard Deviation: {data[:8760].CO_2_eq.std()}')
print('-'*50)
print(f'Covariance: {data[:8760].Price.corr(data[:8760].CO_2_eq)}')